In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gmean
from pandas import DataFrame


In [2]:
df_Results = pd.read_pickle("UsableData/Results.pkl")
df_Results.head(5)

,Market Cap,Employees,Book Value,Equal,Random,Variance,Sales,Custom
1995-12-31,1,1,1,1,1,1,1,1
1996-01-31,1.055732,1.037344,1.027627,1.00793,1.02216,0.975108,1.025296,1.024893
1996-02-29,1.06463,1.050292,1.042715,1.027357,1.038453,1.001764,1.040898,1.040584
1996-03-31,1.036932,1.029997,1.026332,1.018903,1.02427,1.006524,1.025453,1.025301
1996-04-30,1.043941,1.036461,1.032508,1.024496,1.030285,1.011145,1.03156,1.031397


In [3]:
# Based on 3.1 Finance Introduction
total_cumulative_return = (df_Results.iloc[-1]-df_Results.iloc[1])*100
total_cumulative_return

Market Cap    674.306066
Employees     860.363668
Book Value    894.857084
Equal           794.8897
Random        683.701402
Variance       78.283396
Sales         814.299696
Custom        813.344756
dtype: object

In [4]:
# Based on 3.1 Finance Introduction
mean_yearly_return = (((1+(total_cumulative_return/100)))**(1/10)-1)*100
mean_yearly_return

Market Cap    22.713196
Employees     25.384419
Book Value    25.827645
Equal         24.502179
Random        22.861288
Variance       5.952471
Sales         24.769621
Custom        24.756584
dtype: object

In [5]:
df_Returns_Strat = df_Results.pct_change(fill_method=None)*100
df_Returns_Strat.dropna(axis=0, inplace=True)
df_Returns_Strat.head(5)

,Market Cap,Employees,Book Value,Equal,Random,Variance,Sales,Custom
1996-01-31,5.573161,3.734411,2.762652,0.793015,2.215978,-2.489193,2.529567,2.489315
1996-02-29,0.842857,1.248215,1.468301,1.927407,1.593952,2.733667,1.521711,1.530959
1996-03-31,-2.601658,-1.93236,-1.57121,-0.82286,-1.365724,0.475171,-1.483803,-1.468677
1996-04-30,0.675964,0.627619,0.601806,0.548916,0.587203,0.459045,0.595587,0.594512
1996-05-31,9.309816,9.269007,9.247201,9.202487,9.234861,9.126401,9.241946,9.241038


In [6]:
monthly_vol = np.std(df_Returns_Strat, axis=0)
monthly_vol

Market Cap    5.653683
Employees     5.957906
Book Value    5.813626
Equal         4.544852
Random        4.635194
Variance       4.96243
Sales         5.982169
Custom          4.9764
dtype: object

In [7]:
# Read yearly return of DE 10 Year Bound reported monthly
DE10Y = pd.read_csv("RawData/DE10Y.csv", index_col="Datum")
DE10Y.index = pd.to_datetime(DE10Y.index, dayfirst=True)
DE10Y = DE10Y[(DE10Y.index >= df_Results.index[1]) & (DE10Y.index <= df_Results.index[-1])]
for row in DE10Y.index:
    string = DE10Y.loc[row, "Zuletzt"].strip("%").replace(".", "").replace(",", ".")
    DE10Y.at[row, "Zuletzt"] = float(string)
DE10Y.head(5)

,Zuletzt,Eröffn.,Hoch,Tief,+/- %
Datum,,,,,
2021-12-01,-0.179,"-0,3490","-0,1710","-0,4090","-47,81%"
2021-11-01,-0.343,"-0,1100","-0,0650","-0,3630","264,89%"
2021-10-01,-0.094,"-0,1970","-0,0640","-0,2430","-50,79%"
2021-09-01,-0.191,"-0,3730","-0,1720","-0,4010","-49,74%"
2021-08-01,-0.38,"-0,4590","-0,3760","-0,5240","-17,03%"


In [8]:
# Based on 3.1 Finance Introduction
risk_free_return = np.mean(DE10Y["Zuletzt"])
risk_free_return

2.7818504823151122

In [9]:
# Based on 3.1 Finance Introduction
sharpe_ratio = pd.Series(index=df_Results.columns)
for strategy in mean_yearly_return.index:
    sharpe_ratio[strategy] = (mean_yearly_return[strategy] - risk_free_return) / monthly_vol[strategy]
sharpe_ratio

Market Cap    3.525374
Employees     3.793710
Book Value    3.964100
Equal         4.779106
Random        4.331951
Variance      0.638925
Sales         3.675552
Custom        4.415789
dtype: float64

In [10]:
# maximum monthly drawdown
max_monthly_drawdown = pd.Series(index=df_Results.columns)
for strategy in df_Returns_Strat.columns:
    max_monthly_drawdown[strategy] = min(df_Returns_Strat[strategy])
max_monthly_drawdown

Market Cap   -20.740740
Employees    -24.058432
Book Value   -19.562298
Equal        -15.435536
Random       -15.325528
Variance     -27.235294
Sales        -23.165740
Custom       -17.396675
dtype: float64

In [11]:
positive_month = pd.Series(index=df_Results.columns)
for strategy in df_Returns_Strat.columns:
    returns = df_Returns_Strat[strategy]
    returns = returns[returns > 0]
    positive_month[strategy] = len(returns)
positive_month

Market Cap    189.0
Employees     189.0
Book Value    186.0
Equal         199.0
Random        198.0
Variance      185.0
Sales         188.0
Custom        195.0
dtype: float64

In [12]:
df_Analysis = DataFrame(index=df_Results.columns)
df_Analysis["Total Cumulative Return"] = total_cumulative_return
df_Analysis["Mean Yearly Return"] = mean_yearly_return
df_Analysis["Volatility"] = monthly_vol
df_Analysis["Sharpe Ratio"] = sharpe_ratio
df_Analysis["Max Drawdown"] = max_monthly_drawdown
df_Analysis["Positive Month"] = positive_month
df_Analysis.to_pickle("UsableData/Analysis.pkl")
df_Analysis

,Total Cumulative Return,Mean Yearly Return,Volatility,Sharpe Ratio,Max Drawdown,Positive Month
Market Cap,674.306066,22.713196,5.653683,3.525374,-20.740740,189.0
Employees,860.363668,25.384419,5.957906,3.793710,-24.058432,189.0
Book Value,894.857084,25.827645,5.813626,3.964100,-19.562298,186.0
Equal,794.8897,24.502179,4.544852,4.779106,-15.435536,199.0
Random,683.701402,22.861288,4.635194,4.331951,-15.325528,198.0
Variance,78.283396,5.952471,4.96243,0.638925,-27.235294,185.0
Sales,814.299696,24.769621,5.982169,3.675552,-23.165740,188.0
Custom,813.344756,24.756584,4.9764,4.415789,-17.396675,195.0
